<a href="https://colab.research.google.com/github/tbsoaresvalkms/recipes-ml/blob/master/tensorflow2_0/Aprendizagem_por_Refor%C3%A7o_para_Negocia%C3%A7%C3%A3o_de_A%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Etapa 1: Instalação da bibliotecas


In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 54kB/s 
     |████████████████████████████████| 3.0MB 34.6MB/s 
     |████████████████████████████████| 419kB 47.0MB/s 


In [2]:
!pip install pandas-datareader

## Etapa 2: Importação das bibliotecas

In [3]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [4]:
tf.__version__

'2.0.0-alpha0'

## Etapa 3: Construção da IA para negociação de ações

In [0]:
class AI_Trader():
  
  def __init__(self, state_size, action_space = 3, model_name = "AITrader"):
    # O estado que esta vindo do ambiente
    self.state_size = state_size
    # Acoes possiveis que podemos tomar
    self.action_space = action_space
    # +- O numero de acoes para depois fazer o treinamento.
    self.memory = deque(maxlen = 2000)

    self.model_name = model_name
    
    self.gamma = 0.95
    # param que determina se o agente vai seguir a rede ou escolher aleatorio. 1 quer dizer que vai ser 100% aleatorio, o motivo disso, pois no comeco a rede ainda nao esta treinada, entao precisa ser feito escolhas randomicas
    self.epsilon = 1.0
    # como esse param sera reduzido, a gente precisa coloca um minimo, obs que nao eh 0, entao em determinados momentos ele ira escolher pontos aleatorios
    self.epsilon_final = 0.01
    # esse valor determina o quanto o param de aleatoridade ira cair.
    self.epsilon_decay = 0.995
    self.model = self.model_builder()
    
  def model_builder(self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, activation = "relu", input_dim = self.state_size))
    model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = self.action_space, activation = "linear"))
    model.compile(loss = "mse", optimizer = tf.keras.optimizers.Adam(lr = 0.001))
    return model
  
  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  def batch_train(self, batch_size):
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay
      

In [6]:
random.randrange(3)

0

## Etapa 4: Pré-processamento da base de dados

### Definição de funções auxiliares

#### Sigmoid

In [0]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [8]:
sigmoid(0.5)

0.6224593312018546

#### Formatação de preços

In [0]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [10]:
stocks_price_format(100)

'$ 100.000000'

#### Carregador da base de dados

In [0]:
dataset = data_reader.DataReader("AAPL", data_source = "yahoo")

In [12]:
dataset.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-12-09,270.799988,264.910004,270.000000,266.920013,32010600.0,266.920013
2019-12-10,270.070007,265.859985,268.600006,268.480011,22605100.0,268.480011
2019-12-11,271.100006,268.500000,268.809998,270.769989,19689200.0,270.769989
2019-12-12,272.559998,267.320007,267.779999,271.459991,34327600.0,271.459991
2019-12-13,275.299988,270.929993,271.459991,275.149994,33396900.0,275.149994


In [13]:
str(dataset.index[0]).split()[0]

'2010-01-04'

In [14]:
dataset.index[-1]

Timestamp('2019-12-13 00:00:00')

In [15]:
dataset['Close']

Date
2010-01-04     30.572857
2010-01-05     30.625713
2010-01-06     30.138571
2010-01-07     30.082857
2010-01-08     30.282858
                 ...    
2019-12-09    266.920013
2019-12-10    268.480011
2019-12-11    270.769989
2019-12-12    271.459991
2019-12-13    275.149994
Name: Close, Length: 2505, dtype: float64

In [0]:
def dataset_loader(stock_name):
  dataset = data_reader.DataReader(stock_name, data_source = "yahoo")
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close

### Criador de estados

In [17]:
0 - 5 + 1

-4

In [18]:
20 - 5 + 1

16

In [19]:
dataset[16:21]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-27,30.082857,28.504286,29.549999,29.697144,430642100.0,25.839514
2010-01-28,29.357143,28.385714,29.275715,28.469999,293375600.0,24.771774
2010-01-29,28.885714,27.178572,28.725714,27.437143,311488100.0,23.873083
2010-02-01,28.000000,27.328571,27.481428,27.818571,187469100.0,24.204962
2010-02-02,28.045713,27.625713,27.987143,27.980000,174585600.0,24.345423


In [0]:
def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep + 1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
    
  return np.array([state]), windowed_data

### Carregando a base de dados

In [0]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [0]:
s, w = state_creator(data, 1000, 5)

In [23]:
s

array([[0.35368976, 0.28875709, 0.65733224, 0.95302395]])

In [24]:
w

Date
2013-12-17    79.284286
2013-12-18    78.681427
2013-12-19    77.779999
2013-12-20    78.431427
2013-12-23    81.441429
Name: Close, dtype: float64

## Etapa 5: Treinando a IA

### Configuração dos hyper parâmetros

In [0]:
window_size = 10
episodes = 1000
batch_size = 32
data_samples = len(data) - 1

In [26]:
data_samples

2504

### Definição do modelo

In [0]:
trader = AI_Trader(window_size)

In [28]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Loop de treinamento

In [0]:
for episode in range(1, episodes + 1):
  print("Episode: {}/{}".format(episode, episodes))
  state = state_creator(data, 0, window_size + 1)
  total_profit = 0
  trader.inventory = []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data, t + 1, window_size + 1)
    reward = 0
    
    if action == 1 and len(trader.inventory) == 0: # Comprando uma ação
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
    elif action == 2 and len(trader.inventory) > 0: # Vendendo uma ação  
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("Total profit: {}".format(total_profit))
      print("########################")
      
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
     
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    


  0%|          | 0/2504 [00:00<?, ?it/s]

Episode: 1/1000



  0%|          | 5/2504 [00:01<11:46,  3.54it/s]

AI Trader bought:  $ 30.015715



 11%|█         | 272/2504 [01:11<09:20,  3.98it/s]

...


 34%|███▍      | 859/2504 [03:48<07:22,  3.72it/s]

AI Trader bought:  $ 64.187141



 34%|███▍      | 862/2504 [03:49<07:33,  3.62it/s]

AI Trader sold:  $ 63.115715  Profit: - $ 1.071426



 35%|███▍      | 866/2504 [03:50<07:09,  3.81it/s]